In [ ]:
!pip install "shapely<2.0.0"
!pip install google-cloud-aiplatform --upgrade

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import vertexai
from google.cloud import aiplatform

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)
vertexai.init(project=PROJECT_ID, location=LOCATION)

print(f"Vertex AI SDK version: {aiplatform.__version__}")


## PaLM Prompt

In [ ]:
from vertexai.language_models import TextGenerationModel

parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 0.9,
    "top_p": 1
}
model = TextGenerationModel.from_pretrained("text-bison")
response = model.predict(
    """Who was Grace Hopper""",
    **parameters
)
print(f"Response from Model: {response.text}")

## PaLM Streaming

In [ ]:
responses = model.predict_streaming("Tell me about Steve Jobs",**parameters)

for response in responses:
        print(response.text)

## PaLM Chat

In [ ]:
from vertexai.language_models import ChatModel, InputOutputTextPair

parameters = {
    "max_output_tokens": 1024,
    "temperature": 0.9,
    "top_p": 1
}
chat_model = ChatModel.from_pretrained("chat-bison")

chat = chat_model.start_chat(
    context="""""",
)

response = chat.send_message("""Who was Steve Jobs""", **parameters)
print(f"Response from Model: {response.text}")

In [ ]:
response = chat.send_message("""Did he have any children?""", **parameters)
print(f"Response from Model: {response.text}")

In [ ]:
print(chat.message_history)

## Streaming Chat

In [ ]:
chat = chat_model.start_chat()

parameters = {
    "max_output_tokens": 1024,
    "temperature": 0.9,
    "top_p": 1
}
responses = chat.send_message_streaming("""Hi""", **parameters)
for response in responses:
        print(response.text)

In [ ]:
responses = chat.send_message_streaming("""Tell me about Grace Hopper""", **parameters)
for response in responses:
        print(response.text)

In [ ]:
responses = chat.send_message_streaming("""List some key dates in her life""", **parameters)
for response in responses:
        print(response.text)

## Safety Attributes

In [ ]:
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_p": 1,
    "top_k": 40
}

model = TextGenerationModel.from_pretrained("text-bison")

response = model.predict(
    """Tell me a joke about puppies""",
    **parameters
)
print(response.text)
print(response.safety_attributes)
print("---------------------")


response = model.predict(
    """Tell me a joke about politics""",
    **parameters
)
print(response.text)
print(response.safety_attributes)